In [2]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
import pandas as pd
import json
import time

In [3]:
def car_info(link):
    cars = requests.get(link,'html.parser')
    soup = bs(cars.content)
    coloumns = []
    info = []
    #Getting all the info from the information boxes
    for i in soup.select('th',class_= 'css-1pusycv e155vlc0'):
        coloumns.append(i.text)
    for j in soup.select('td',class_ ='attributes-table-value'):
        info.append(j.get_text('',strip=True))
    dictionary = dict(zip(coloumns,info)) #zipping the file and converting it into dict
    #Getting the price of the car
    price = soup.find(class_ = 'css-bk9fzy e1pt9h6u5')
    dictionary['Price'] = price.get_text('',strip=True)
    #Extracting the name of the car
    title = soup.select('h1',class_ ='css-1uk1gs8 e1pt9h6u3')
    title = str(title)
    title = title.split('>')
    dictionary['Name'] = title[1]
    #Extracting the location
    location = soup.find(class_ = 'css-23ot5d e1pt9h6u4')
    dictionary['Location'] = location.get_text('',strip = True)
    return dictionary

In [ ]:
tabs = [] #There are alltogether 50 tabs on the whole 5 wheeler cars and 30 sublinks on each of them
for i in range(2,51):
    tabs.append('page{}'.format(i))

In [5]:
pages_base_link = 'https://www.gumtree.com/cars/uk/page2?vehicle_seats=5'
final_webpages = [] 
for j in tabs:
    all_webpages.append(pages_base_link.replace('page2',j)) 

In [7]:
#Extracting all the sublinks from the links
all_subwebpages = [] #Extracting all the sublinks and collecting in the lists
for i in final_webpages:
    try:
        baselink = 'https://www.gumtree.com'
        cars = requests.get(i,'html.parser')
        soup = bs(cars.content)
        for j in soup.select('article',class_='clearfix list-listing-maxi'):
            link = j.find('a',href = True)
            all_subwebpages.append(baselink+link['href'])
    except:
        print('Error Found')

In [6]:
all_subwebpages.remove('https://www.gumtree.com') #Removing the links that contains

In [7]:
#Saving the sublink list as txt file so that we don't have to scrape it again
with open("carlist.txt", "w") as output:
    output.write(str(all_subwebpages))

In [ ]:
dataset = [] #Extracting all the data from the sublinks and adding here
counter = 0
for i in all_subwebpages:
    try:
        dataset.append(car_info(i))
        counter+=1
        print('Extracted links',counter)
    except:
        print('Errors found')

In [9]:
#Saving all the data in json format
def save_data(title,data):
    with open(title,'w', encoding = 'utf-8') as f:
        json.dump(data,f,ensure_ascii = False,indent = 2)
#SAVING
save_data('gumtree.json',dataset) #Saving the dataset list with dictionary into a json file 